In [1]:
from Pooling import Pooling_2D_density_3D
from Conv_Layer import Conv_RBS_density_I2_3D
import torch
import gc
import torch.nn as nn  # the neural network library of pytorch
import load_dataset_letao as load  # module with function to load MNIST
from toolbox import reduce_MNIST_dataset
from training import test_net, train_net
from Dense import Dense_RBS_density_3D
from toolbox import Basis_Change_I_to_HW_density_3D, Trace_out_dim, get_reduced_layers_structure, map_HW_to_measure, get_full_pyramid_gates
from training import train_net_stride
import warnings
warnings.simplefilter('ignore')

In [2]:
class QCNN(nn.Module):
    def __init__(self, I, O, J, K, k, device):
        super(QCNN, self).__init__()
        list_gates_pyramid = get_reduced_layers_structure(O+J, 5)
        list_gates_pyramid_small = get_full_pyramid_gates(5)
            
        self.conv1 = Conv_RBS_density_I2_3D(I,K,J,device)
        self.pool1 = Pooling_2D_density_3D(I, O, J, device)
        self.conv2 = Conv_RBS_density_I2_3D(O,K,J,device)
        self.pool2 = Pooling_2D_density_3D(O, O//2, J, device)
        self.basis_map = Basis_Change_I_to_HW_density_3D(O//2, J, k, device)
        self.dense1 = Dense_RBS_density_3D(O//2, J, k, list_gates_pyramid, device)
        self.tomo = Trace_out_dim(10, device)
        self.dense2 = Dense_RBS_density_3D(0, 5, k, list_gates_pyramid_small, device)

    def forward(self, x):
        c1 = self.conv1(x)
        p1 = self.pool1(c1)
        c2 = self.conv2(p1)
        p2 = self.pool2(c2)
        b1 = self.basis_map(p2)
        d1 = self.dense1(b1)
        to = self.tomo(d1)
        d2 = self.dense2(to)
        output = map_HW_to_measure(d2, device)
        return output    # return x for visualization

In [3]:


I = 12 # dimension of image we use
O = I//2 # dimension after pooling
J = 2 # number of channel
k = 3
K = 2 # size of kernel
batch_size = 10 # batch number
scala = 6000 # time we reduce dataset
learning_rate = 1e-1
device = torch.device("mps")

train_loader, test_loader, dim_in, dim_out = load.load_MNIST(batch_size=batch_size)
reduced_loader = reduce_MNIST_dataset(train_loader, scala)
reduced_test_loader = reduce_MNIST_dataset(test_loader, 100)

conv_network = QCNN(I, O, J, K, k, device)
optimizer = torch.optim.Adam(conv_network.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

loss_list = []
accuracy_list = []
for epoch in range(10):
    train_loss, train_accuracy = train_net_stride(batch_size, I, J, k, conv_network, train_loader, criterion, optimizer, device)
    loss_list.append(train_loss)
    accuracy_list.append(train_accuracy*100)
    print(f'Epoch {epoch}: Loss = {train_loss:.6f}, accuracy = {train_accuracy*100:.4f} %')
    
    if epoch%2 == 0:
        test_loss, test_accuracy = test_net(batch_size, I, J, k, conv_network, reduced_test_loader, criterion, device)
        print(f'Evaluation on test set: Loss = {test_loss:.6f}, accuracy = {test_accuracy*100:.4f} %')

Epoch 0: Loss = 2.627540, accuracy = 0.0000 %
Evaluation on test set: Loss = 2.451135, accuracy = 8.0000 %
Epoch 1: Loss = 2.429156, accuracy = 10.0000 %
Epoch 2: Loss = 2.401943, accuracy = 10.0000 %
Evaluation on test set: Loss = 2.416550, accuracy = 8.0000 %
Epoch 3: Loss = 2.379159, accuracy = 10.0000 %
Epoch 4: Loss = 2.302812, accuracy = 10.0000 %
Evaluation on test set: Loss = 2.400648, accuracy = 7.0000 %
Epoch 5: Loss = 2.234006, accuracy = 10.0000 %
Epoch 6: Loss = 2.188027, accuracy = 10.0000 %
Evaluation on test set: Loss = 2.388007, accuracy = 14.0000 %
Epoch 7: Loss = 2.207392, accuracy = 20.0000 %
Epoch 8: Loss = 2.250664, accuracy = 20.0000 %
Evaluation on test set: Loss = 2.377637, accuracy = 14.0000 %
Epoch 9: Loss = 2.266976, accuracy = 20.0000 %


In [15]:
test_loss, test_accuracy = test_net(batch_size, I, J, k, conv_network, reduced_test_loader, criterion, device)
print(f'Evaluation on test set: Loss = {test_loss:.6f}, accuracy = {test_accuracy*100:.4f} %')

Evaluation on test set: Loss = 2.321713, accuracy = 14.0000 %
